**<h2> Importing Libraries**

In [18]:
import os
import tensorflow as tf
import numpy as np

**<h2> Preprocessing using InceptionV3**

First we will get the location of the images we want to specify and store it in a list.Here, the list will be 
**img_coco_path**.

In [15]:
PATH=r'/home/tarushi/Desktop/Img-Caption/images/train2014'

In [3]:
img_list=os.listdir(PATH)
img_list[:10]

['COCO_train2014_000000105728.jpg',
 'COCO_train2014_000000175053.jpg',
 'COCO_train2014_000000047249.jpg',
 'COCO_train2014_000000264241.jpg',
 'COCO_train2014_000000162693.jpg',
 'COCO_train2014_000000204785.jpg',
 'COCO_train2014_000000203976.jpg',
 'COCO_train2014_000000223461.jpg',
 'COCO_train2014_000000057300.jpg',
 'COCO_train2014_000000147250.jpg']

In [16]:
PATH=r'/home/tarushi/Desktop/Img-Caption/images/train2014/'
img_coco_path=[]
for i in img_list:
    img_coco_path.append(PATH+i)
len(img_coco_path)

16383

We will only be using 3000 images in this case.

In [6]:
img_coco_path=img_coco_path[:3000]

The load function will do the following:
* Read the image into the img variable.
* Transform the jpeg image into a tensor with 3 channels(RGB)
* Resize the image such that it can be feeded into the inception v3 network
* Preprocess the image using InceptionV3
* Return the image and it's path

In [7]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

Next,we'll be loading the InceptionV3 without the last layer , as we'll be using it to extract features from our images.

In [8]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)


87916544/87910968 [==============================] - 163s 2us/step


In [9]:
image_features_extract_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In the next few line of codes, we'll be making a tensorflow dataset out of the given set of images.
This dataset then will undergo preprocessing , followed by parallel loading (using tf.data.experimental.AUTOTUNE).All of this will happen in a batch of 16 images at once.
Next we would extract the features from the images and store them in batch_features.

In [11]:
# Get unique images
encode_train = sorted(set(img_coco_path))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,(batch_features.shape[0], -1, batch_features.shape[3]))


In [13]:
batch_features.shape

TensorShape([8, 64, 2048])

In [14]:
batch_features[:2]

<tf.Tensor: shape=(2, 64, 2048), dtype=float32, numpy=
array([[[0.        , 0.06971237, 0.        , ..., 0.012679  ,
         0.03764814, 1.1014669 ],
        [0.        , 0.        , 0.29454818, ..., 0.02173308,
         0.47982717, 0.9880909 ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.87323695, 0.75990623],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.75797486, 0.        ],
        [0.        , 0.        , 0.30268586, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.77979195, 0.17871454, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.03424155],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.13294414],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.6755341 , ..., 0. 

In the next line of code, we map the location with the extracted features and save it as a numpy ndarray.

In [28]:
for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())